# SQL Joins Explained for Beginners

This notebook demonstrates the different types of SQL joins using dummy data. We will use two tables: `Employees` and `Departments`, to visualize how data is combined.

In [ ]:
import sqlite3
import pandas as pd

## 1. Setup Dummy Data

We will create an in-memory SQLite database and populate it with sample data designed to highlight the differences between join types.

In [ ]:
# Create an in-memory SQLite database
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Create Departments Table
cursor.execute('''
CREATE TABLE Departments (
    dept_id INTEGER PRIMARY KEY,
    dept_name TEXT
)
''')

# Create Employees Table
cursor.execute('''
CREATE TABLE Employees (
    emp_id INTEGER PRIMARY KEY,
    emp_name TEXT,
    dept_id INTEGER,
    FOREIGN KEY (dept_id) REFERENCES Departments (dept_id)
)
''')

# Insert Data into Departments
# We have HR and Engineering. Marketing is a department with NO employees initially.
departments_data = [
    (1, 'HR'),
    (2, 'Engineering'),
    (3, 'Marketing')
]
cursor.executemany('INSERT INTO Departments VALUES (?, ?)', departments_data)

# Insert Data into Employees
# Alice and Bob interact with existing departments.
# Charlie has a NULL dept_id (not assigned).
# David is assigned to dept_id 99 (a department that doesn't exist in the Departments table - strictly speaking, FK constraints might block this, but SQLite allows it by default/or we use it for demo).
employees_data = [
    (101, 'Alice', 1),    # Matches HR
    (102, 'Bob', 2),      # Matches Engineering
    (103, 'Charlie', None), # No Department
    (104, 'David', 99)    # Invalid Department
]
cursor.executemany('INSERT INTO Employees VALUES (?, ?, ?)', employees_data)

conn.commit()

### View the Tables

In [ ]:
print("--- Employees Table ---")
df_emp = pd.read_sql_query("SELECT * FROM Employees", conn)
display(df_emp)

print("\n--- Departments Table ---")
df_dept = pd.read_sql_query("SELECT * FROM Departments", conn)
display(df_dept)

## 2. INNER JOIN

**Definition**: Returns records that have matching values in both tables.

- Alice (Dept 1) matches HR (Dept 1).
- Bob (Dept 2) matches Engineering (Dept 2).
- Charlie, David, and Marketing are excluded because they don't have a match in the other table.

In [ ]:
query = '''
SELECT Employees.emp_name, Departments.dept_name
FROM Employees
INNER JOIN Departments ON Employees.dept_id = Departments.dept_id;
'''
pd.read_sql_query(query, conn)

## 3. LEFT JOIN (or LEFT OUTER JOIN)

**Definition**: Returns all records from the left table (`Employees`), and the matched records from the right table (`Departments`). If there is no match, the result is NULL on the right side.

- Alice and Bob get their departments.
- Charlie (NULL dept) and David (Dept 99) are included, but `dept_name` will be None/NaN because there is no matching department.

In [ ]:
query = '''
SELECT Employees.emp_name, Departments.dept_name
FROM Employees
LEFT JOIN Departments ON Employees.dept_id = Departments.dept_id;
'''
pd.read_sql_query(query, conn)

## 4. CROSS JOIN

**Definition**: Returns the Cartesian product of the set of records from the two tables. Warning: This can create very large results!

- Combines every employee with every department.

In [ ]:
query = '''
SELECT Employees.emp_name, Departments.dept_name
FROM Employees
CROSS JOIN Departments;
'''
pd.read_sql_query(query, conn)

## 5. FULL OUTER JOIN (Simulated)

**Definition**: Return all records when there is a match in either left or right table.

*Note: SQLite does not support `FULL OUTER JOIN` directly, but we can simulate it by unioning a LEFT JOIN and a RIGHT JOIN (or simulating the Right Join by swapping tables).*

The result includes:
- Matches (Alice, Bob)
- Left-only (Charlie, David)
- Right-only (Marketing)

In [ ]:
# Simulate FULL OUTER JOIN
query = '''
SELECT Employees.emp_name, Departments.dept_name
FROM Employees
LEFT JOIN Departments ON Employees.dept_id = Departments.dept_id

UNION ALL

SELECT Employees.emp_name, Departments.dept_name
FROM Departments
LEFT JOIN Employees ON Departments.dept_id = Employees.dept_id
WHERE Employees.dept_id IS NULL;
'''
pd.read_sql_query(query, conn)

## Summary Table

| Join Type | Description | Result in our Example |
|-----------|-------------|-----------------------|
| **INNER** | Overlap only | Alice (HR), Bob (Eng) |
| **LEFT** | All Left + Overlap | Alice, Bob, Charlie (Null), David (Null) |
| **RIGHT** | All Right + Overlap | Alice, Bob, Marketing (Null Emp) |
| **FULL** | Everything | Everyone + Every Dept |

In [ ]:
# Close the connection
conn.close()